In [1]:
!pip install spacy

In [2]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.1/12.8 MB 825.8 kB/s eta 0:00:16
     ---------------------------------------- 0.1/12.8 MB 1.2 MB/s eta 0:00:11
      --------------------------------------- 0.3/12.8 MB 1.6 MB/s eta 0:00:08
     - -------------------------------------- 0.3/12.8 MB 1.7 MB/s eta 0:00:08
     - -------------------------------------- 0.4/12.8 MB 1.7 MB/s eta 0:00:08
     - -------------------------------------- 0.6/12.8 MB 1.8 MB/s eta 0:00:07
     -- ------------------------------------- 0.7/12.8 MB 2.0 MB/s eta 0:00:07
     -- ------------------------------------- 0.8/12.8 MB 2.1 MB/s eta 0:00:06
     -- ------------------------------------- 0.9/12.8 MB 2.0 MB/s eta 0:00:06
     --- ------------------------------------ 1.0/12.8 MB 2.1 MB/s eta 0:00:06
     --- ------------------------------------ 1.2/12.8 MB 2.2 MB/

In [3]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import spacy
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

C:\Users\Sarin\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
nltk.download('punkt') ## Download NLTK resources
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sarin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sarin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
# Load the dataset
data = pd.read_csv("BBC_DATA.csv")

In [6]:
data

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business
...,...,...,...
1485,857,double eviction from big brother model caprice...,entertainment
1486,325,dj double act revamp chart show dj duo jk and ...,entertainment
1487,1590,weak dollar hits reuters revenues at media gro...,business
1488,1587,apple ipod family expands market apple has exp...,tech


In [7]:
data.isnull().sum() #checking for null

ArticleId    0
Text         0
Category     0
dtype: int64

In [8]:
data.info()  # checking info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ArticleId  1490 non-null   int64 
 1   Text       1490 non-null   object
 2   Category   1490 non-null   object
dtypes: int64(1), object(2)
memory usage: 35.1+ KB


In [9]:
data.drop('ArticleId',axis = 1, inplace = True)  #droping ID

In [10]:
sample_article = data['Text'][0] ## Sample news article

In [11]:
sample_article

'worldcom ex-boss launches defence lawyers defending former worldcom chief bernie ebbers against a battery of fraud charges have called a company whistleblower as their first witness.  cynthia cooper  worldcom s ex-head of internal accounting  alerted directors to irregular accounting practices at the us telecoms giant in 2002. her warnings led to the collapse of the firm following the discovery of an $11bn (£5.7bn) accounting fraud. mr ebbers has pleaded not guilty to charges of fraud and conspiracy.  prosecution lawyers have argued that mr ebbers orchestrated a series of accounting tricks at worldcom  ordering employees to hide expenses and inflate revenues to meet wall street earnings estimates. but ms cooper  who now runs her own consulting business  told a jury in new york on wednesday that external auditors arthur andersen had approved worldcom s accounting in early 2001 and 2002. she said andersen had given a  green light  to the procedures and practices used by worldcom. mr ebb

In [12]:
word_tokens = word_tokenize(sample_article) ## Tokenization using NLTK
sent_tokens = sent_tokenize(sample_article)

In [13]:
word_tokens

['worldcom',
 'ex-boss',
 'launches',
 'defence',
 'lawyers',
 'defending',
 'former',
 'worldcom',
 'chief',
 'bernie',
 'ebbers',
 'against',
 'a',
 'battery',
 'of',
 'fraud',
 'charges',
 'have',
 'called',
 'a',
 'company',
 'whistleblower',
 'as',
 'their',
 'first',
 'witness',
 '.',
 'cynthia',
 'cooper',
 'worldcom',
 's',
 'ex-head',
 'of',
 'internal',
 'accounting',
 'alerted',
 'directors',
 'to',
 'irregular',
 'accounting',
 'practices',
 'at',
 'the',
 'us',
 'telecoms',
 'giant',
 'in',
 '2002.',
 'her',
 'warnings',
 'led',
 'to',
 'the',
 'collapse',
 'of',
 'the',
 'firm',
 'following',
 'the',
 'discovery',
 'of',
 'an',
 '$',
 '11bn',
 '(',
 '£5.7bn',
 ')',
 'accounting',
 'fraud',
 '.',
 'mr',
 'ebbers',
 'has',
 'pleaded',
 'not',
 'guilty',
 'to',
 'charges',
 'of',
 'fraud',
 'and',
 'conspiracy',
 '.',
 'prosecution',
 'lawyers',
 'have',
 'argued',
 'that',
 'mr',
 'ebbers',
 'orchestrated',
 'a',
 'series',
 'of',
 'accounting',
 'tricks',
 'at',
 'worldcom

In [14]:
sent_tokens

['worldcom ex-boss launches defence lawyers defending former worldcom chief bernie ebbers against a battery of fraud charges have called a company whistleblower as their first witness.',
 'cynthia cooper  worldcom s ex-head of internal accounting  alerted directors to irregular accounting practices at the us telecoms giant in 2002. her warnings led to the collapse of the firm following the discovery of an $11bn (£5.7bn) accounting fraud.',
 'mr ebbers has pleaded not guilty to charges of fraud and conspiracy.',
 'prosecution lawyers have argued that mr ebbers orchestrated a series of accounting tricks at worldcom  ordering employees to hide expenses and inflate revenues to meet wall street earnings estimates.',
 'but ms cooper  who now runs her own consulting business  told a jury in new york on wednesday that external auditors arthur andersen had approved worldcom s accounting in early 2001 and 2002. she said andersen had given a  green light  to the procedures and practices used by w

In [15]:
porter_stemmer = PorterStemmer()  ## Stemming and Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

In [16]:
stemmed_words = [porter_stemmer.stem(word) for word in word_tokens]
lemmatized_words = [wordnet_lemmatizer.lemmatize(word) for word in word_tokens]

In [17]:
stemmed_words

['worldcom',
 'ex-boss',
 'launch',
 'defenc',
 'lawyer',
 'defend',
 'former',
 'worldcom',
 'chief',
 'berni',
 'ebber',
 'against',
 'a',
 'batteri',
 'of',
 'fraud',
 'charg',
 'have',
 'call',
 'a',
 'compani',
 'whistleblow',
 'as',
 'their',
 'first',
 'wit',
 '.',
 'cynthia',
 'cooper',
 'worldcom',
 's',
 'ex-head',
 'of',
 'intern',
 'account',
 'alert',
 'director',
 'to',
 'irregular',
 'account',
 'practic',
 'at',
 'the',
 'us',
 'telecom',
 'giant',
 'in',
 '2002.',
 'her',
 'warn',
 'led',
 'to',
 'the',
 'collaps',
 'of',
 'the',
 'firm',
 'follow',
 'the',
 'discoveri',
 'of',
 'an',
 '$',
 '11bn',
 '(',
 '£5.7bn',
 ')',
 'account',
 'fraud',
 '.',
 'mr',
 'ebber',
 'ha',
 'plead',
 'not',
 'guilti',
 'to',
 'charg',
 'of',
 'fraud',
 'and',
 'conspiraci',
 '.',
 'prosecut',
 'lawyer',
 'have',
 'argu',
 'that',
 'mr',
 'ebber',
 'orchestr',
 'a',
 'seri',
 'of',
 'account',
 'trick',
 'at',
 'worldcom',
 'order',
 'employe',
 'to',
 'hide',
 'expens',
 'and',
 'infla

In [18]:
lemmatized_words

['worldcom',
 'ex-boss',
 'launch',
 'defence',
 'lawyer',
 'defending',
 'former',
 'worldcom',
 'chief',
 'bernie',
 'ebbers',
 'against',
 'a',
 'battery',
 'of',
 'fraud',
 'charge',
 'have',
 'called',
 'a',
 'company',
 'whistleblower',
 'a',
 'their',
 'first',
 'witness',
 '.',
 'cynthia',
 'cooper',
 'worldcom',
 's',
 'ex-head',
 'of',
 'internal',
 'accounting',
 'alerted',
 'director',
 'to',
 'irregular',
 'accounting',
 'practice',
 'at',
 'the',
 'u',
 'telecom',
 'giant',
 'in',
 '2002.',
 'her',
 'warning',
 'led',
 'to',
 'the',
 'collapse',
 'of',
 'the',
 'firm',
 'following',
 'the',
 'discovery',
 'of',
 'an',
 '$',
 '11bn',
 '(',
 '£5.7bn',
 ')',
 'accounting',
 'fraud',
 '.',
 'mr',
 'ebbers',
 'ha',
 'pleaded',
 'not',
 'guilty',
 'to',
 'charge',
 'of',
 'fraud',
 'and',
 'conspiracy',
 '.',
 'prosecution',
 'lawyer',
 'have',
 'argued',
 'that',
 'mr',
 'ebbers',
 'orchestrated',
 'a',
 'series',
 'of',
 'accounting',
 'trick',
 'at',
 'worldcom',
 'ordering'

In [19]:
nlp = spacy.load('en_core_web_sm')    ## Load SpaCy's English model

In [38]:
sample_article = data['Text'][0]    ### Sample news article

In [39]:
doc = nlp(sample_article)     # Perform & Visualize  named entity recognition

from spacy import displacy
displacy.render(doc, style='dep', jupyter=True)

In [40]:
doc = nlp(sample_article)

from spacy import displacy
displacy.render(doc, style='ent', jupyter=True)

In [23]:
tokenized_data = [word_tokenize(article) for article in data['Text']]  
## Tokenize the entire dataset

In [24]:
# Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_data, 
vector_size=100, window=5, min_count=1, workers=4)

In [25]:
# Get vector representation of a sample word
sample_word = 'boss'
print("Vector representation of '{}' using Word2Vec:".format(sample_word))
print(word2vec_model.wv[sample_word])

Vector representation of 'boss' using Word2Vec:
[-3.95847768e-01  3.83343518e-01 -9.78021100e-02  6.66937649e-01
  8.40373635e-02 -9.14380550e-01 -1.50691301e-01  8.11718822e-01
 -2.61483997e-01  7.67028853e-02 -3.36084783e-01 -3.76266718e-01
  2.25746363e-01  2.82062441e-01  7.70480871e-01 -5.04985392e-01
  5.19475102e-01 -4.37354267e-01  9.71327007e-01 -4.95445967e-01
  4.98229593e-01  6.33540750e-02  3.71071547e-01 -3.48587990e-01
  1.91988736e-01 -1.98239759e-01 -1.03342628e+00  8.17626357e-01
 -4.95773017e-01  2.20464855e-01  1.02113438e+00 -2.49197595e-02
  1.19351201e-01 -6.55615568e-01 -1.27519697e-01  4.86395299e-01
  1.82172313e-01 -3.07630777e-01 -4.10733595e-02 -4.79090780e-01
 -5.25738113e-02 -5.77002227e-01 -5.09302676e-01  3.65464717e-01
  3.22293371e-01  3.39584649e-01 -6.74517035e-01  3.61958630e-02
  1.59005269e-01 -1.99691862e-01  7.20387280e-01 -4.90644753e-01
 -2.03298494e-01 -7.51988143e-02 -1.87247097e-01  8.61383080e-01
  4.76918697e-01  5.81359982e-01 -7.056108

In [26]:
# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

In [27]:
# Fit and transform the dataset using the fitted vectorizer
tfidf_matrix = tfidf_vectorizer.fit_transform(data['Text'])

In [28]:
# Calculate cosine similarity between two news articles
# Let's calculate similarity between the first two articles as an example
similarity_score = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])
print("Cosine Similarity between the first two news articles:", similarity_score[0][0])

Cosine Similarity between the first two news articles: 0.07875931547482325
